# Vikas Gogia (300292869)

## Imports

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=ec5034b9013fc688a1502ee08adc34e44cbbf9e61fcde02d0791daa06eb6b4f6
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [ ]:
from google.colab import drive
from sentence_transformers import SentenceTransformer, models, InputExample, losses
from torch import nn
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
import pandas as pd
import torch
import numpy as np

## Gdrive

In [ ]:
# mount drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


## Train data

In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/train.data', sep="\t", header = None)

In [ ]:
train = train.iloc[:800, 2:5]
train.columns = ["sen1", "sen2", "score"]

In [ ]:
train['score'] = train['score'].astype('str').apply(lambda s: s[1])
train['score'] = train['score'].astype('float32').apply(lambda s: s/5)
train['isParaphrase'] = np.where(train['score'] >= 0.5, True, False)

In [ ]:
train.head()

,sen1,sen2,score,isParaphrase
0,EJ Manuel the 1st QB to go in this draft,But my bro from the 757 EJ Manuel is the 1st Q...,1.0,True
1,EJ Manuel the 1st QB to go in this draft,Can believe EJ Manuel went as the 1st QB in th...,1.0,True
2,EJ Manuel the 1st QB to go in this draft,EJ MANUEL IS THE 1ST QB what,0.6,True
3,EJ Manuel the 1st QB to go in this draft,EJ da 1st QB off da board,0.4,False
4,EJ Manuel the 1st QB to go in this draft,Manuel is the 1st QB to get drafted,0.8,True


## Dev Data

In [ ]:
dev = pd.read_csv('/content/gdrive/MyDrive/dev.data', sep="\t", header = None)

In [ ]:
dev = dev.iloc[:100, 2:5]
dev.columns = ["sen1", "sen2", "score"]

In [ ]:
dev['score'] = dev['score'].astype('str').apply(lambda s: s[1])
dev['score'] = dev['score'].astype('float32').apply(lambda s: s/5)
dev['isParaphrase'] = np.where(dev['score'] >= 0.5, True, False)

In [ ]:
dev.head()

,sen1,sen2,score,isParaphrase
0,A Walk to Remember is the definition of true love,A Walk to Remember is on and Im in town and Im...,0.2,False
1,A Walk to Remember is the definition of true love,A Walk to Remember is the cutest thing,0.6,True
2,A Walk to Remember is the definition of true love,A walk to remember is on ABC family youre welcome,0.2,False
3,A Walk to Remember is the definition of true love,A walk to remember is so amazing and inspiring,0.6,True
4,A Walk to Remember is the definition of true love,BUT GUYS ITS ON MY FAVE PART OF A WALK TO REME...,0.2,False


## Test Data

In [ ]:
test = pd.read_csv('/content/gdrive/MyDrive/test.data', sep="\t", header = None)

In [ ]:
test = test.iloc[:100, 2:4]
test.columns = ["sen1", "sen2"]
test.head()

,sen1,sen2
0,All the home alones watching 8 mile,8 mile is on thats my movie
1,All the home alones watching 8 mile,The last rap battle in 8 Mile nevr gets old ahah
2,All the home alones watching 8 mile,The rap battle at the end of 8 mile gets me so...
3,The Ending to 8 Mile is my fav part of the who...,Rabbit on 8 mile out of place but determined t...
4,The Ending to 8 Mile is my fav part of the who...,See 8 Mile is always on but it s the tv versio...


## Approach 1 (Baseline Algorithm)

In [ ]:
def baselineStringMatcher(senList1, senList2):
  op = []
  for i in range(len(senList1)):
    op.append(senList1[i] == senList2[i])

  return op 

In [ ]:
y_pred = baselineStringMatcher(dev['sen1'], dev['sen2'])

In [ ]:
print("Precision: ", precision_score(dev['isParaphrase'], y_pred))
print("Recall   : ", recall_score(dev['isParaphrase'], y_pred))
print("F1 score : ", f1_score(dev['isParaphrase'], y_pred))

Precision:  1.0
Recall   :  0.04
F1 score :  0.07692307692307693


## Evaluation

In [ ]:
def cosine(u, v):    
  return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
def evaluateBertModel(model, senList1, senList2):
  s1_embed = model.encode(senList1)
  s2_embed = model.encode(senList2)
  
  y_pred = []

  for i in range(len(s1_embed)):
    cos_sim_score = cosine(s1_embed[i], s2_embed[i])
    y_pred.append(cos_sim_score >= 0.5)

  print("Precision: ", precision_score(dev['isParaphrase'], y_pred))
  print("Recall   : ", recall_score(dev['isParaphrase'], y_pred))
  print("F1 score : ", f1_score(dev['isParaphrase'], y_pred))

## Approach 2 (BERT)

In [ ]:
sen1 = train['sen1']
sen2 = train['sen2']
scores = train['score']

In [ ]:
word_embedding_model = models.Transformer('bert-base-uncased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_examples = []

In [ ]:
train['sen1'] = train['sen1'].astype('str')
train['sen2'] = train['sen2'].astype('str')
train['score'] = train['score'].astype('float32')

In [ ]:
for i in range(len(sen1)):
  train_examples.append(InputExample(texts=[sen1[i], sen2[i]], label = scores[i]))

train_dataloader = DataLoader(train_examples, shuffle = True, batch_size = 16)

In [ ]:
train_loss = losses.CosineSimilarityLoss(model.double())

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs = 5, warmup_steps = 100)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
evaluateBertModel(model, dev['sen1'], dev['sen2'])

Precision:  0.4666666666666667
Recall   :  0.56
F1 score :  0.509090909090909


## Approach 3 (Modified BERT)

In [ ]:
word_embedding_model2 = models.Transformer('distilbert-base-uncased')
pooling_model2 = models.Pooling(word_embedding_model2.get_word_embedding_dimension())
dense_model = models.Dense(in_features = pooling_model2.get_sentence_embedding_dimension(), out_features = 768, activation_function = nn.Tanh())

model2 = SentenceTransformer(modules=[word_embedding_model2, pooling_model2, dense_model])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_loss2 = losses.CosineSimilarityLoss(model2.double())

In [ ]:
model2.fit(train_objectives=[(train_dataloader, train_loss2)], epochs = 5, warmup_steps = 100)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
evaluateBertModel(model2, dev['sen1'], dev['sen2'])

Precision:  0.3673469387755102
Recall   :  0.72
F1 score :  0.4864864864864865


##Test Eval (Approach 2 - BERT is best)

In [ ]:
def bertTestPred(model, senList1, senList2):
  s1_embed = model.encode(senList1)
  s2_embed = model.encode(senList2)
  
  y_pred = []

  for i in range(len(s1_embed)):
    cos_sim_score = cosine(s1_embed[i], s2_embed[i])
    y_pred.append(cos_sim_score >= 0.5)

  return y_pred

In [ ]:
y_pred = bertTestPred(model, test['sen1'], test['sen2'])

In [ ]:
test['isParaphrasePred'] = y_pred

In [ ]:
test

,sen1,sen2,isParaphrasePred
0,All the home alones watching 8 mile,8 mile is on thats my movie,False
1,All the home alones watching 8 mile,The last rap battle in 8 Mile nevr gets old ahah,False
2,All the home alones watching 8 mile,The rap battle at the end of 8 mile gets me so...,False
3,The Ending to 8 Mile is my fav part of the who...,Rabbit on 8 mile out of place but determined t...,False
4,The Ending to 8 Mile is my fav part of the who...,See 8 Mile is always on but it s the tv versio...,False
...,...,...,...
95,Rafa Benitez I must thank you,I am so happy for RAFA BENITEZ VictorMoses and...,True
96,Rafa Benitez I must thank you,Pleased for Benitez hasnt deserved the stick h...,False
97,Rafa Benitez I must thank you,Once a red always a blue rafa Benitez we want you,False
98,THANK YOU SO MUCH RAFA BENITEZ,Benitez is alright tho man fuck chelsea fans t...,False
